In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/10/25 05:32:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_green = spark.read.parquet('data_one/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

In [5]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [6]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    

    SUM(total_amount) amount,
    COUNT(1) AS number_records  
    
FROM
    green
WHERE
    lpep_pickup_datetime =  '2020-01-01 00:00:00'
    
GROUP BY
    1, 2
ORDER BY 
    1 , 2
""")

In [7]:
df_green_revenue.show()

+----+----+------+--------------+
|hour|zone|amount|number_records|
+----+----+------+--------------+
+----+----+------+--------------+



In [9]:
df_green_revenue.write.parquet('data_one/report/revenue/green', mode='overwrite')

In [10]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [11]:
df_yellow = spark.read.parquet('data_one/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [12]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    

    SUM(total_amount) amount,
    COUNT(1) AS number_records  
    
FROM
    yellow
WHERE
    tpep_pickup_datetime =  '2020-01-01 00:00:00'
    
GROUP BY
    1, 2
ORDER BY 
    1 , 2
""")

In [13]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data_one/report/revenue/yellow', mode = 'overwrite')